In [2]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2.credentials import Credentials
from google.oauth2 import service_account
from google.auth.transport.requests import Request
import os
import os.path
import json
import pickle
import requests

def oauth():
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 
            #   'https://www.googleapis.com/auth/script.projects', 'https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/drive.file'
              ]
    creds_filename = 'test-daebong-service-account.json'  # 서비스 계정 파일 경로를 지정합니다.

    # 서비스 계정 파일을 사용하여 인증 정보를 로드합니다.
    creds = service_account.Credentials.from_service_account_file(creds_filename, scopes=SCOPES)
    return creds

def oauthByWeb():
    # 필요한 스코프를 지정합니다.
    SCOPES = [
        'https://www.googleapis.com/auth/spreadsheets',
        'https://www.googleapis.com/auth/script.projects',
        'https://www.googleapis.com/auth/drive',
        'https://www.googleapis.com/auth/drive.file'
    ]
    creds = None
    # 'token.json' 파일이 존재하면, 저장된 인증 정보를 불러옵니다.
    if os.path.exists('token.json'):
        with open('token.json', 'rb') as token:
            creds = pickle.load(token)

    # 저장된 인증 정보가 없거나, 유효하지 않은 경우 새로운 인증을 진행합니다.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            creds_filename = 'oauth-new-daebong.json'
            flow = InstalledAppFlow.from_client_secrets_file(creds_filename, SCOPES)
            creds = flow.run_local_server(port=8080)
        # 새로운 인증 정보를 'token.json'에 저장합니다.
        with open('token.json', 'wb') as token:
            pickle.dump(creds, token)

    return creds



In [8]:
def column_index(col):
    # 엑셀 스타일의 컬럼명을 0기반 인덱스로 변환
    num = 0
    for c in col:
        if c in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ':
            num = num * 26 + (ord(c) - ord('A') + 1)
    return num - 1

In [3]:
def validate_and_correct_ranges(ranges, sheet_id):
    corrected_ranges = []
    for range_ in ranges:
        start_col, end_col = range_.split(':')
        # 열 인덱스 계산을 위한 함수 호출
        start_col_index = column_index(start_col)
        end_col_index = column_index(end_col) + 1
        # 잘못된 범위가 있는지 검증
        if start_col_index < end_col_index:
            corrected_ranges.append({
                'sheetId':sheet_id,
                'startRowIndex': 3,  # 4행부터 시작 (0 인덱스)
                'endRowIndex': 300,  # 200행까지
                'startColumnIndex': start_col_index,
                'endColumnIndex': end_col_index
            })
    return corrected_ranges

In [7]:
def get_values_from_range(service, spreadsheet_id, range_name):
    # Google Sheets API를 사용하여 특정 범위의 값 가져오기
    result = service.spreadsheets().values().get(spreadsheetId=spreadsheet_id, range=range_name).execute()
    # 값이 있는지 확인 후 반환
    values = result.get('values', [])

    return values
    

In [9]:
def main(service_sheets, SPREADSHEET_ID, SHEET_TITLE, SHEET_TITLE_PRODUCT):
    sheet_metadata = service_sheets.spreadsheets().get(spreadsheetId=SPREADSHEET_ID).execute()
    sheets = sheet_metadata.get('sheets', '')

    # 제목이 일치하는 시트의 ID 찾기
    sheet_id = None
    sheet_id_product = None
    for sheet in sheets:
        if sheet['properties']['title'] == SHEET_TITLE:
            sheet_id = sheet['properties']['sheetId']
            continue

        if sheet['properties']['title'] == SHEET_TITLE_PRODUCT:
            sheet_id_product = sheet['properties']['sheetId']
            continue

    if sheet_id is not None:
        print(f'Sheet ID for "{SHEET_TITLE}" is {sheet_id}')
    if sheet_id_product is not None:
        print(f'Sheet ID for "{SHEET_TITLE_PRODUCT}" is {sheet_id_product}')
    else:
        print(f'No sheet found with title "{SHEET_TITLE}"')

    # 스프레드시트 ID 설정
    

    # 필요한 범위 리스트로 정의
    ranges = [
        'J4:O200', 'Q4:V200', 'X4:AC200', 'AE4:AJ200', 
        'AL4:AQ200', 'AS4:AX200', 'AZ4:BE200', 'BG4:BL200', 
        'BN4:BS200', 'BU4:BZ200'
    ]

    # 병합 요청 정보 생성
    # requests = []
    # for range_ in ranges:
    #     requests.append({
    #         'mergeCells': {
    #             'range': {
    #                 'sheetId': sheet_id,
    #                 'startRowIndex': 3,
    #                 'endRowIndex': 200,
    #                 'startColumnIndex': column_index(range_[0]),
    #                 'endColumnIndex': column_index(range_.split(':')[1]) + 1,
    #             },
    #             'mergeType': 'MERGE_ALL'
    #         }
    #     })
    # 수정된 범위를 사용하여 병합 요청 생성
    corrected_requests = []
    for corrected_range in validate_and_correct_ranges(ranges, sheet_id):
        corrected_requests.append({
            'mergeCells': {
                'range': corrected_range,
                'mergeType': 'MERGE_ALL'
            }
        })

      # J2 셀에 하이퍼링크 추가
    # hyperlink_formula = '=HYPERLINK("https://docs.google.com/spreadsheets/d/1p0WM4X2JztS1LfGKVdUXd3GTvBJJGYPNO0ya8ihfns4/edit#gid=1594213233","공지사항 더 보러가기")'
    # corrected_requests.append({
    #     'updateCells': {
    #         'rows': [{'values': [{'userEnteredValue': {'formulaValue': hyperlink_formula}}]}],
    #         'fields': 'userEnteredValue',
    #         'start': {
    #             'sheetId': sheet_id,
    #             'rowIndex': 1,  # 2행
    #             'columnIndex': 9  # J열
    #         }
    #     }
    # })

        # 'Sheet2'의 P10 셀에 텍스트 추가
    instruction_text = "상품목록 검색 방법: 컨트롤+f 동시 입력 후 검색창에 '복숭아' 입력시 복숭아만 검색 가능합니다"
    corrected_requests.append({
        'updateCells': {
            'rows': [{'values': [{'userEnteredValue': {'stringValue': instruction_text}}]}],
            'fields': 'userEnteredValue',
            'start': {
                'sheetId': sheet_id_product,
                'rowIndex': 9,  # 10행
                'columnIndex': 15  # P열
            }
        }
    })

    # 병합 요청 실행
    # request_body = {'requests': requests}
    request_body = {'requests': corrected_requests}
    request = service_sheets.spreadsheets().batchUpdate(spreadsheetId=SPREADSHEET_ID, body=request_body).execute()
    print('Cells merged successfully.')


In [10]:
import time
import random
user_sheet_id = "1Ajz2g-pChLSoies5umhPMcgTp5m2MIjhljJOviPlWCU"
# spreadsheet_copy_id = "1p0WM4X2JztS1LfGKVdUXd3GTvBJJGYPNO0ya8ihfns4"

# SPREADSHEET_ID = '1RODBFbJ0p49qvpEeMOLLBtllLLUKZkxKX0Dax4V_-ms'
SHEET_TITLE = '공지사항'
SHEET_TITLE_PRODUCT = '상품목록'

if __name__ == "__main__":
    creds = oauthByWeb()
    service_sheets = build('sheets', 'v4', credentials=creds)
    service_drive = build("drive", "v3", credentials=creds) 
    range_name = "셀러컨택포인트!A475:T503"
    
    user_data_list = get_values_from_range(service_sheets, user_sheet_id, range_name)

    user_ids = [[item[1], item[10]] for item in user_data_list]

    for spreadsheet_id in user_ids:
        time.sleep(random.uniform(4, 7))
        if spreadsheet_id == '':
            continue
        print(spreadsheet_id[0], "apply_formula_to_cell_sheet2")
        main(service_sheets, spreadsheet_id[1], SHEET_TITLE, SHEET_TITLE_PRODUCT)
       

   


493 apply_formula_to_cell_sheet2
Sheet ID for "공지사항" is 1005496708
Sheet ID for "상품목록" is 35482029
Cells merged successfully.
494 apply_formula_to_cell_sheet2
Sheet ID for "공지사항" is 1005496708
Sheet ID for "상품목록" is 35482029
Cells merged successfully.
495 apply_formula_to_cell_sheet2
Sheet ID for "공지사항" is 1005496708
Sheet ID for "상품목록" is 35482029
Cells merged successfully.
496 apply_formula_to_cell_sheet2
Sheet ID for "공지사항" is 1005496708
Sheet ID for "상품목록" is 35482029
Cells merged successfully.
497 apply_formula_to_cell_sheet2
Sheet ID for "공지사항" is 1005496708
Sheet ID for "상품목록" is 35482029
Cells merged successfully.
498 apply_formula_to_cell_sheet2
Sheet ID for "공지사항" is 1005496708
Sheet ID for "상품목록" is 35482029
Cells merged successfully.
499 apply_formula_to_cell_sheet2
Sheet ID for "공지사항" is 1005496708
Sheet ID for "상품목록" is 35482029
Cells merged successfully.
500 apply_formula_to_cell_sheet2
Sheet ID for "공지사항" is 1005496708
Sheet ID for "상품목록" is 35482029
Cells merged succes